## 공통: Import 및 로더/전처리 함수

In [ ]:

import open3d as o3d
import numpy as np

def load_and_process_ply(path, scale=100.0, shift_x=0.0, rotate_deg_x=0.0):
    """
    PLY 파일을 로드하고, 스케일/이동/회전을 적용합니다.
    - scale: 좌표 단위 환산용 (예: m→mm 환산 전 임시 100배 등)
    - shift_x: X축 방향 평행이동
    - rotate_deg_x: X축 기준 회전 (deg)
    """
    mesh = o3d.io.read_triangle_mesh(path)
    if mesh.is_empty():
        raise ValueError(f"메쉬가 비어있습니다: {path}")
    mesh.compute_vertex_normals()
    mesh.scale(scale, center=(0, 0, 0))
    mesh.translate((shift_x, 0, 0))
    if rotate_deg_x != 0.0:
        radians = np.deg2rad(rotate_deg_x)
        R = mesh.get_rotation_matrix_from_axis_angle([radians, 0, 0])
        mesh.rotate(R, center=(0, 0, 0))
    return mesh


## 경로 / 변환 파라미터 설정

In [ ]:

# 입력 PLY 경로
PATH1 = "../assets/ply/mesh_merged_imp_gpu_impeller_pred2.ply"  # 복원 메쉬
PATH2 = "../assets/ply/0807_normalized.ply"      # 원본 메쉬

# 공통 전처리 파라미터
SCALE      = 100.0      # 임시 스케일 (후에 mm 환산 복원)
SHIFT_X    = 6.95       # X축 평행이동 (필요 시 0.0으로)
ROT_X_DEG1 = 11.8       # 복원 메쉬의 X축 회전 (deg)
ROT_X_DEG2 = 0.0        # 원본 메쉬의 X축 회전 (deg)

# 로드 & 전처리
mesh1 = load_and_process_ply(PATH1, scale=SCALE, shift_x=SHIFT_X, rotate_deg_x=ROT_X_DEG1)
mesh2 = load_and_process_ply(PATH2, scale=SCALE, shift_x=0.0,     rotate_deg_x=ROT_X_DEG2)

print("메쉬 로드 완료:")
print(" - mesh1(복원) 삼각형:", np.asarray(mesh1.triangles).shape[0])
print(" - mesh2(원본)  삼각형:", np.asarray(mesh2.triangles).shape[0])


# 절대 위치 / 상대 위치 / 길이 측정

In [ ]:
import numpy as np
import open3d as o3d

def _deg_from_rotmat(R):
    # 두 회전행렬 사이 각도(라디안→도)
    # (수치오차로 trace가 살짝 벗어날 수 있어 clip)
    t = np.clip((np.trace(R) - 1) / 2, -1.0, 1.0)
    return np.degrees(np.arccos(t))

def compute_absolute_position_accuracy(mesh1, mesh2, scale=SCALE):
    aabb1, aabb2 = mesh1.get_axis_aligned_bounding_box(), mesh2.get_axis_aligned_bounding_box()
    min_diff_mm = (aabb1.get_min_bound() - aabb2.get_min_bound()) / scale
    max_diff_mm = (aabb1.get_max_bound() - aabb2.get_max_bound()) / scale

    c1 = np.asarray(aabb1.get_center())
    c2 = np.asarray(aabb2.get_center())
    trans_vec_mm = (c1 - c2) / scale

    # 추가: 표 기준용 평균/최대 오차(축 기준)
    abs_mean_mm = np.mean(np.abs(trans_vec_mm))
    abs_max_mm  = np.max(np.abs(trans_vec_mm))

    # 기존: 축평균 RMS (참고용)
    trans_rmse_mm = np.sqrt(np.mean(trans_vec_mm**2))

    # 자세(참고)
    obb1, obb2 = mesh1.get_oriented_bounding_box(), mesh2.get_oriented_bounding_box()
    R1, R2 = obb1.R, obb2.R
    R_rel = R2.T @ R1
    rot_err_deg = _deg_from_rotmat(R_rel)

    return {
        "translation_vector_mm": trans_vec_mm,
        "translation_rmse_mm": trans_rmse_mm,
        "abs_mean_mm": abs_mean_mm,   # ▶ 절대 위치 평균오차(축 기준)
        "abs_max_mm":  abs_max_mm,    # ▶ 절대 위치 최대오차(축 기준)
        "aabb_min_diff_mm": min_diff_mm,
        "aabb_max_diff_mm": max_diff_mm,
        "rotation_error_deg": rot_err_deg
    }

def compute_relative_length_accuracy(mesh1, mesh2, scale=SCALE, use_obb=True):
    """
    상대 길이(크기) 정확도: 위치/자세 무시하고 '치수'만 비교
    - 기본: OBB의 세 변 길이 비교(회전 불변, 형상 자오선 기준)
    - 옵션: AABB 치수 비교(간단하지만 회전에 민감)
    """
    if use_obb:
        b1, b2 = mesh1.get_oriented_bounding_box(), mesh2.get_oriented_bounding_box()
        size1, size2 = b1.extent, b2.extent            # [Lx, Ly, Lz]
    else:
        b1, b2 = mesh1.get_axis_aligned_bounding_box(), mesh2.get_axis_aligned_bounding_box()
        size1, size2 = b1.get_extent(), b2.get_extent()

    size1_mm = size1 / scale
    size2_mm = size2 / scale
    diff_mm  = np.abs(size1_mm - size2_mm)

    # 면적 비교(동일 스케일 복원 규칙)
    area1_mm2 = mesh1.get_surface_area() / (scale * scale)
    area2_mm2 = mesh2.get_surface_area() / (scale * scale)

    out = {
        "size1_mm": size1_mm,            # [Lx, Ly, Lz]
        "size2_mm": size2_mm,
        "size_abs_diff_mm": diff_mm,     # per-axis |Δ|
        "area1_mm2": area1_mm2,
        "area2_mm2": area2_mm2
    }
    return out

def print_absolute_position_report(res_abs,
                                   thresh_mean_mm=0.10,  # 표: 평균 0.1mm
                                   thresh_max_mm=0.30):  # 표: 최대 0.3mm
    dx, dy, dz = res_abs["translation_vector_mm"]
    print("\n— 절대 위치 정확도 —")
    print(f"중심 오프셋 [mm]: dx={dx:.4f}, dy={dy:.4f}, dz={dz:.4f}")
    print(f"평균오차[mm](축기준): {res_abs['abs_mean_mm']:.4f} → {'o' if res_abs['abs_mean_mm']<=thresh_mean_mm else '❌'} (≤{thresh_mean_mm}mm)")
    print(f"최대오차[mm](축기준): {res_abs['abs_max_mm']:.4f} → {'o' if res_abs['abs_max_mm']<=thresh_max_mm else '❌'} (≤{thresh_max_mm}mm)")
    print(f"(참고) RMS[mm]: {res_abs['translation_rmse_mm']:.4f}")
    mn = res_abs["aabb_min_diff_mm"]; mx = res_abs["aabb_max_diff_mm"]
    print(f"AABB min 코너 차이[mm]: {mn[0]:.4f}, {mn[1]:.4f}, {mn[2]:.4f}")
    print(f"AABB max 코너 차이[mm]: {mx[0]:.4f}, {mx[1]:.4f}, {mx[2]:.4f}")

def compute_relative_position_accuracy_icp(mesh1, mesh2, scale=SCALE,
                                           n_points=20000, icp_thresh_mm=1.0):
    # 1) 메쉬 → 포인트샘플
    pcd1 = mesh1.sample_points_uniformly(number_of_points=n_points)  # ← fix
    pcd2 = mesh2.sample_points_uniformly(number_of_points=n_points)  # ← fix

    # 2) 초기 정렬: AABB 중심 일치
    T0 = np.eye(4)
    T0[:3, 3] = (mesh2.get_axis_aligned_bounding_box().get_center()
                 - mesh1.get_axis_aligned_bounding_box().get_center())
    pcd1.transform(T0)  # in-place

    # 3) ICP (point-to-point)
    threshold = icp_thresh_mm * scale
    result = o3d.pipelines.registration.registration_icp(
        pcd1, pcd2, threshold, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    pcd1.transform(result.transformation)

    # 4) 정합 후 잔차 (양방향)
    d12 = np.asarray(pcd1.compute_point_cloud_distance(pcd2)) / scale  # mm
    d21 = np.asarray(pcd2.compute_point_cloud_distance(pcd1)) / scale  # mm
    d = np.concatenate([d12, d21])

    return {
        "rel_mean_mm": float(np.mean(d)),
        "rel_max_mm":  float(np.max(d)),
    }

def print_relative_length_report(res_len, threshold_len_mm=0.05, label="(OBB 치수)"):
    d = res_len["size_abs_diff_mm"]
    print("\n— 상대 길이(크기) 정확도 —", label)
    print(f"크기1 [mm] : {res_len['size1_mm'][0]:.3f}, {res_len['size1_mm'][1]:.3f}, {res_len['size1_mm'][2]:.3f}")
    print(f"크기2 [mm] : {res_len['size2_mm'][0]:.3f}, {res_len['size2_mm'][1]:.3f}, {res_len['size2_mm'][2]:.3f}")
    print(f"|Δ|   [mm] : {d[0]:.4f}, {d[1]:.4f}, {d[2]:.4f}")
    marks = ["o" if x <= threshold_len_mm else "❌" for x in d]
    print(f"축별 기준(±{threshold_len_mm}mm): {' '.join(marks)}")
    print(f"면적 비교  : A={res_len['area1_mm2']:.3f} mm² / B={res_len['area2_mm2']:.3f} mm²")

def print_relative_position_report(res_rel, mean_thr=0.05, max_thr=0.10):
    print("\n— 상대 위치 정확도(정합 후 잔차) —")
    m, M = res_rel["rel_mean_mm"], res_rel["rel_max_mm"]
    print(f"평균오차[mm]: {m:.4f} → {'o' if m<=mean_thr else '❌'} (≤{mean_thr}mm)")
    print(f"최대오차[mm]: {M:.4f} → {'o' if M<=max_thr else '❌'} (≤{max_thr}mm)")


In [ ]:


res_abs = compute_absolute_position_accuracy(mesh1, mesh2, scale=SCALE)
print_absolute_position_report(res_abs, thresh_mean_mm=0.10, thresh_max_mm=0.30)  # 표 기준

res_rel = compute_relative_position_accuracy_icp(mesh1, mesh2, scale=SCALE, n_points=20000, icp_thresh_mm=1.0)
print_relative_position_report(res_rel, mean_thr=0.05, max_thr=0.10)               # 표 기준

res_len_obb  = compute_relative_length_accuracy(mesh1, mesh2, scale=SCALE, use_obb=True)
print_relative_length_report(res_len_obb,  threshold_len_mm=0.05, label="(OBB 치수)")

# 기하학적 정확도

In [ ]:

def compute_rmse_distance(mesh1, mesh2, n_points=10000):
    pcd1 = mesh1.sample_points_uniformly(number_of_points=n_points)
    pcd2 = mesh2.sample_points_uniformly(number_of_points=n_points)
    d1 = np.asarray(pcd1.compute_point_cloud_distance(pcd2))
    d2 = np.asarray(pcd2.compute_point_cloud_distance(pcd1))
    rmse = np.sqrt((np.mean(d1 ** 2) + np.mean(d2 ** 2)) / 2)
    return rmse

def compute_geometric_accuracy(mesh1, mesh2, n_points=10000):
    pcd1 = mesh1.sample_points_uniformly(number_of_points=n_points)
    pcd2 = mesh2.sample_points_uniformly(number_of_points=n_points)
    d1 = np.asarray(pcd1.compute_point_cloud_distance(pcd2))
    d2 = np.asarray(pcd2.compute_point_cloud_distance(pcd1))
    chamfer = (np.mean(d1) + np.mean(d2)) / 2
    hausdorff = max(np.max(d1), np.max(d2))
    return chamfer, hausdorff

rmse = compute_rmse_distance(mesh1, mesh2)
chamfer, hausdorff = compute_geometric_accuracy(mesh1, mesh2)

# 스케일 복원 (길이계열 → ÷SCALE)
rmse_mm      = rmse / SCALE
chamfer_mm   = chamfer / SCALE
hausdorff_mm = hausdorff / SCALE

print("— 기하학적 정확도 —")
print(f"RMSE              : {rmse_mm:.4f} mm")
print(f"Chamfer Distance  : {chamfer_mm:.4f} mm → {'o 기준 만족' if chamfer_mm <= 0.1 else '❌ 기준 초과'}")
print(f"Hausdorff Distance: {hausdorff_mm:.4f} mm → {'o 기준 만족' if hausdorff_mm <= 0.2 else '❌ 기준 초과'}")


# 표면 일치도

In [ ]:

def compute_surface_matching_accuracy(source_mesh, target_mesh, n_points=10000):
    pcd_s = source_mesh.sample_points_uniformly(number_of_points=n_points)
    pcd_t = target_mesh.sample_points_uniformly(number_of_points=n_points)
    d = np.asarray(pcd_s.compute_point_cloud_distance(pcd_t))
    mean_error = np.mean(d)
    max_error  = np.max(d)
    return mean_error, max_error

mean_surface, max_surface = compute_surface_matching_accuracy(mesh1, mesh2)
mean_surface_mm = mean_surface / SCALE
max_surface_mm  = max_surface  / SCALE

print("— 표면 일치도 —")
print(f"평균 오차: {mean_surface_mm:.4f} mm (참고용)")
print(f"최대 오차: {max_surface_mm:.4f} mm → {'o' if max_surface_mm <= 0.1 else '❌'}")


# 특정특징 정확도

In [ ]:
# %% [markdown]
# # x=0 평면 단면(YZ 투영) 추출

# %%
import os
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

# 이미 위에서 정의된 것들 재사용 (PATH1/2, SCALE, SHIFT_X, ROT_X_DEG1/2, load_and_process_ply, mesh1/mesh2)
# 여기서는 mesh1, mesh2가 존재한다고 가정

# 단면 파라미터
X_PLANE_VALUE_MM = 0.0        # x=0 평면 (mm 기준)
SLICE_THICKNESS_MM = 0.5      # ± thickness/2 (mm)
N_SAMPLE = 300_000            # 표면 샘플 포인트 수 (충분히 크게)

# 저장 경로
OUT_DIR = "../out/sections"
os.makedirs(OUT_DIR, exist_ok=True)

print("SCALE =", SCALE)


In [ ]:
# %%
def sample_points_from_mesh(mesh, n=N_SAMPLE):
    """메쉬 표면 포인트 균일 샘플링"""
    return mesh.sample_points_uniformly(number_of_points=int(n))

def slice_by_plane_X(points_xyz, x0, thickness_mm, scale):
    """
    x = x0 (mm) 평면 주변 두께(±thickness/2)로 슬라이스.
    내부 좌표가 mm×SCALE라면, mm → 내부단위 변환해서 마스크 생성.
    """
    # 내부 좌표 단위에서의 평면/두께
    x0_internal = x0 * scale
    half_band_internal = (thickness_mm * scale) / 2.0
    x = points_xyz[:, 0]
    mask = np.abs(x - x0_internal) <= half_band_internal
    return mask

def project_to_yz(points_xyz, scale):
    """
    3D 포인트(mm×SCALE)를 YZ 평면 2D(mm)로 투영.
    (y,z)만 뽑고 SCALE로 나눠 mm로 환산.
    """
    yz = points_xyz[:, 1:3] / scale
    return yz

def plot_section(yz, title, save_path=None, s=1.0):
    if yz.size == 0:
        print("[경고] 표시할 포인트가 없습니다.")
        return
    plt.figure(figsize=(6,6))
    plt.scatter(yz[:,0], yz[:,1], s=s)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title(title)
    plt.xlabel("Y [mm]"); plt.ylabel("Z [mm]")
    plt.grid(True, alpha=0.2)
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"[저장] {save_path}")
    plt.show()

def save_points(yz, path_csv):
    if yz.size == 0:
        print("[정보] 비어있는 단면이라 저장을 건너뜁니다.")
        return
    np.savetxt(path_csv, yz, fmt="%.6f", delimiter=",")
    print(f"[저장] {path_csv}  (columns: Y_mm,Z_mm)")


In [ ]:
# %%
pcd1 = sample_points_from_mesh(mesh1, N_SAMPLE)
pcd2 = sample_points_from_mesh(mesh2, N_SAMPLE)

pts1 = np.asarray(pcd1.points)  # 내부단위 (mm×SCALE 가정)
pts2 = np.asarray(pcd2.points)

print("샘플 포인트수:",
      "mesh1:", pts1.shape[0],
      "mesh2:", pts2.shape[0])


In [ ]:
# %%
mask1 = slice_by_plane_X(pts1, x0=X_PLANE_VALUE_MM, thickness_mm=SLICE_THICKNESS_MM, scale=SCALE)
mask2 = slice_by_plane_X(pts2, x0=X_PLANE_VALUE_MM, thickness_mm=SLICE_THICKNESS_MM, scale=SCALE)

sec1_yz = project_to_yz(pts1[mask1], scale=SCALE)
sec2_yz = project_to_yz(pts2[mask2], scale=SCALE)

print(f"mesh1 단면점: {sec1_yz.shape[0]}  |  mesh2 단면점: {sec2_yz.shape[0]}")
if sec1_yz.size == 0 or sec2_yz.size == 0:
    print("[주의] 단면 포인트가 너무 적거나 없음. SLICE_THICKNESS_MM를 키워보세요 (예: 1.0~2.0 mm).")


In [ ]:
# %%
# mesh1 (복원)
csv1 = os.path.join(OUT_DIR, "mesh1_x0_yz_section.csv")
save_points(sec1_yz, csv1)
plot_section(sec1_yz, title="mesh1 @ x=0 (YZ section)", save_path=None, s=1.0)

# mesh2 (원본)
csv2 = os.path.join(OUT_DIR, "mesh2_x0_yz_section.csv")
save_points(sec2_yz, csv2)
plot_section(sec2_yz, title="mesh2 @ x=0 (YZ section)", save_path=None, s=1.0)


# 부피 및 치수 정확도

In [ ]:
# 겉넓이와 최대 지름 기반 구 근사 부피 계산

# 겉넓이 (mm^2)
surface_area1 = mesh1.get_surface_area() / (SCALE ** 2)
surface_area2 = mesh2.get_surface_area() / (SCALE ** 2)

# 최대 지름(=최대 거리) 계산 (mm)
from scipy.spatial.distance import pdist

verts1_mm = np.asarray(mesh1.vertices) / SCALE
verts2_mm = np.asarray(mesh2.vertices) / SCALE

diameter1 = np.max(pdist(verts1_mm))
diameter2 = np.max(pdist(verts2_mm))

# 구로 근사: 겉넓이 S = 4πr^2, 지름 d = 2r → r = d/2
# 부피 V = (4/3)πr^3
def sphere_volume_from_area_diameter(area, diameter):
    r = diameter / 2
    return (4/3) * np.pi * r**3

volume1_est = sphere_volume_from_area_diameter(surface_area1, diameter1)
volume2_est = sphere_volume_from_area_diameter(surface_area2, diameter2)

print("— 겉넓이·지름 기반 구 근사 부피 —")
print(f"mesh1(복원): {volume1_est:.6f} mm³")
print(f"mesh2(원본): {volume2_est:.6f} mm³")
print(f"차이: {abs(volume1_est - volume2_est):.6f} mm³")

# 형상 편차 시각화

In [ ]:
import plotly.graph_objects as go

def visualize_o3d_mesh(title, mesh):
    # open3d mesh의 vertex/face 추출
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    fig = go.Figure(
        data=[
            go.Mesh3d(
                x=vertices[:, 0],
                y=vertices[:, 1],
                z=vertices[:, 2],
                i=triangles[:, 0],
                j=triangles[:, 1],
                k=triangles[:, 2],
                opacity=1.0,
                color='lightblue'
            )
        ],
        layout=dict(
            title=dict(text=title, x=0.5),
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False),
                aspectmode='data'
            ),
            margin=dict(t=40, b=10, l=0, r=0),
        )
    )
    fig.show()

# mesh1, mesh2 시각화
visualize_o3d_mesh("mesh1 (복원)", mesh1)
visualize_o3d_mesh("mesh2 (원본)", mesh2)